In [1]:
import argparse
from sentence_transformers import SentenceTransformer, InputExample, losses, evaluation
from torch.utils.data import DataLoader
import random
import pandas as pd

In [13]:
def generate_samples(df, anchor_column="question", positive_cols=[], negative_cols=[], use_inbatch=False, max_triplets_per_sample=-1):
    """
    generates the triplets from the dataframe
    :param df: the dataframe
    :param anchor_column: the column name of the anchor column
    :param positive_cols: the columns that are used to generate the positive samples
    :param negative_cols: the columns that are used to generate the negative samples
    :param use_inbatch: if true, the samples are generated in batches, otherwise, they are present in the negative_cols
    :param max_triplets_per_sample: the maximum number of triplets per sample

    :return: a list of the triplets
    """
    if not use_inbatch and len(negative_cols) == 0: raise ValueError("if use_inbatch is false, negative_cols must be specified")
    if use_inbatch and len(negative_cols) > 0: raise ValueError("if use_inbatch is true, negative_cols must not be specified")

    train_samples = []
    num_triplets_per_sample = max(len(positive_cols), len(negative_cols))
    # if max_triplets_per_sample is specified, we limit the number of triplets per sample
    if max_triplets_per_sample > 0: num_triplets_per_sample = min(num_triplets_per_sample, max_triplets_per_sample) 

    for idx, row in df.iterrows():
        anchor_text = row[anchor_column]
        positives, negatives = [], []

        # generate the positive samples
        if len(positive_cols) >= num_triplets_per_sample:
            positive_cols_chosen = random.sample(positive_cols, num_triplets_per_sample)
        else:
            positive_cols_chosen = random.choices(positive_cols, k=num_triplets_per_sample)
        positives = [row[col] for col in positive_cols_chosen]

        # generate the negative samples
        if use_inbatch:
            negative_idxs_list = list(df.index.difference([idx]))
            negative_idxs = random.sample(negative_idxs_list, num_triplets_per_sample)
            negatives = [df.loc[idx, anchor_column] for idx in negative_idxs]
        else:
            if len(negative_cols) >= num_triplets_per_sample:
                negative_cols_chosen = random.sample(negative_cols, num_triplets_per_sample)
            else:
                negative_cols_chosen = random.choices(negative_cols, k=num_triplets_per_sample)
            negatives = [row[col] for col in negative_cols_chosen]

        for positive, negative in zip(positives, negatives):
            print(anchor_text, positive, negative)
            train_samples.append(InputExample(anchor_text, positive, negative))

    return train_samples

In [3]:
def train(train_samples, val_samples, model_path='sentence-transformers/paraphrase-mpnet-base-v2', num_epochs=10, 
        batch_size=16, output_dir='output', verbose=True):
    """
    train the biencoder
    :param train_samples: the training samples
    :param val_samples: the validation samples
    :param model_path: the path to the pretrained model
    :param num_epochs: the number of epochs
    :param batch_size: the batch size
    :param output_dir: the output directory
    :param verbose: if true, the training is printed

    :return: the trained model
    """
    if verbose:
        print(f"[INFO] Training Set Size: {len(train_samples)}")
        print(f"[INFO] Validation Set Size: {len(val_samples)}")

    # load the model
    model = SentenceTransformer(model_path)

    # Make the dataloaders
    train_dataloader = DataLoader(train_samples, batch_size=batch_size, shuffle=True)
    evaluator = evaluation.TripletEvaluator.from_input_examples(val_samples)

    warmup_steps = int(len(train_dataloader) * num_epochs * 0.1)
    train_loss = losses.TripletLoss(model=model)

    if verbose:
        print(f"[INFO] Training for {num_epochs} epochs")

    model.fit(train_objectives=[(train_dataloader, train_loss)],
            epochs=num_epochs, warmup_steps=warmup_steps, evaluator=evaluator,
            output_path=output_dir)

    if verbose:
        print(f"[INFO] Training finished!")
        print(f"[INFO] Saving model to {output_dir}")

    model.save(output_dir)

In [14]:
train_path = "..\\other\\AquaSSMBA_val.csv"
val_path = "..\\other\\AquaSSMBA_val.csv"
anchor_column = "question"
positive_cols = ["aug_1", "aug_2", "aug_3", "aug_7"]
# negative_cols = ["aug_4", "aug_5", "aug_6"]
negative_cols = []
use_inbatch = True
max_triplets_per_sample = -1


train_df = pd.read_csv(train_path)
val_df = pd.read_csv(val_path)

train_samples = generate_samples(df=train_df, anchor_column=anchor_column, positive_cols=positive_cols, 
                            negative_cols=negative_cols, use_inbatch=use_inbatch, 
                            max_triplets_per_sample=max_triplets_per_sample)

val_samples = generate_samples(df=val_df, anchor_column=anchor_column, positive_cols=positive_cols,
                            negative_cols=negative_cols, use_inbatch=use_inbatch,
                            max_triplets_per_sample=max_triplets_per_sample)

# model = train(train_samples=train_samples, val_samples=val_samples, model_path=args.model_path, num_epochs=args.num_epochs,
#             batch_size=args.batch_size, output_dir=args.output_dir, verbose=args.verbose)


Three birds are flying at a fast rate of 900 kilometers per hour. What is their speed in miles per minute? [1km = 0.6 miles] their birds are flying into a hemisphereif of 900 kilometers per hour. what is their speed in seconds per minute? [ iq 0 = 0. 6 miles ] Alex has enough money to buy 30 bricks. If the bricks each cost 20 cents less, Grace could buy 10 more bricks. How much money does Grace have to spend on bricks?
Three birds are flying at a fast rate of 900 kilometers per hour. What is their speed in miles per minute? [1km = 0.6 miles] the birds are flying to a fast rate of 2 kilometers per hour. what is their speed 100 miles per moment? [ 1km = 2. 6 miles ] John and Mike had equal amount of money in savings at the beginning of last year. Then, Mike’s savings increased by 20% by the end of last year and decreased by 20% this year. John’s savings at first decreased by 20% by the end of last year and then increased by 20% this year. What percentage of John’s and Mike’s savings comb

In [7]:
print(train_samples[:5])

[<sentence_transformers.readers.InputExample.InputExample object at 0x0000026F16F17F88>, <sentence_transformers.readers.InputExample.InputExample object at 0x0000026F16F17848>, <sentence_transformers.readers.InputExample.InputExample object at 0x0000026F16F17E48>, <sentence_transformers.readers.InputExample.InputExample object at 0x0000026F1717C408>, <sentence_transformers.readers.InputExample.InputExample object at 0x0000026F1717C388>]
